In [1]:
import pandas as pd
import math
import heapq
import copy
import time
import random
import numpy as np
from tqdm.notebook import tqdm

In [2]:
###########################
#ノードとリンクのcsvデータ読み込み
###########################

#link[lid, nid1, nid2, 長さ, width]
df_link = pd.read_csv('node_and_link/link.csv', encoding = 'shift-jis')
#node[nid, X, Y, lid1, lid2, lid3, lid4, lid5]
df_node = pd.read_csv('node_and_link/node.csv', encoding = 'shift-jis')

display(df_node.head(10))
display(df_link.head(10))

,nid,X,Y,lid1,lid2,lid3,lid4,lid5
0,1,-19621.7335,-31651.9260,1,87.0,88.0,NaN,NaN
1,2,-19626.0230,-31623.8619,1,2.0,1915.0,NaN,NaN
2,3,-19632.7611,-31573.9943,2,3.0,NaN,NaN,NaN
3,4,-19638.8995,-31531.1167,3,4.0,NaN,NaN,NaN
4,5,-19647.4526,-31467.4674,4,5.0,NaN,NaN,NaN
5,6,-19649.7852,-31434.1494,5,6.0,NaN,NaN,NaN
6,7,-19650.4267,-31421.5240,6,7.0,NaN,NaN,NaN
7,8,-19652.3603,-31383.6698,7,8.0,NaN,NaN,NaN
8,9,-19654.5570,-31350.2412,8,9.0,NaN,NaN,NaN
9,10,-19655.6246,-31333.9097,9,10.0,NaN,NaN,NaN


,lid,nid1,nid2,length,width
0,1,1,2,28.396,4
1,2,2,3,50.331,4
2,3,3,4,43.323,4
3,4,4,5,64.234,4
4,5,5,6,33.406,4
5,6,6,7,12.644,4
6,7,7,8,37.911,4
7,8,8,9,33.507,4
8,9,9,10,16.370,4
9,10,10,11,5.104,4


In [3]:
###########################
#描画用のcanvasサイズと地図サイズの調整
###########################
canvas_size = [1000, 500] #canvasサイズ(仮)
min_x, min_y = df_node['X'].min(), df_node['Y'].min() #最も左上のX,Y座標
max_x, max_y = df_node['X'].max(), df_node['Y'].max() #最も右下X,Y座標
vertex = (min_x, min_y)
original_size = (math.fabs(max_x  - min_x), math.fabs(max_y  - min_y)) #元の地図サイズ
ratio = canvas_size[0] / original_size[0] #canvasサイズの横幅に合わせて地図の拡大倍率を変更
canvas_size[1] = original_size[1] * ratio #地図と同じ縦横比になるようにcanvasサイズの縦を変更

In [4]:
###########################
#ノードとリンクのクラス作成とインスタンス化
###########################

class Node:
    def __init__(self, nid, x, y, link_ids):
        self.id = nid #ノードのID
        self.x, self.y = x, y #座標
        self.link_ids = link_ids #接するリンクIDのリスト
        self.passed_links = [] #経由したリンクIDのリスト
        self.total_d, self.total_d2 =  float('inf'), float('inf') #スタート地点からの距離(+ゴールまでの直線距離)
        
class Link:
    def __init__(self, lid, nid1, nid2, d, w):
        self.id = lid #リンクのID
        self.nid1, self.nid2 = nid1, nid2 #両端ノードのID
        self.d = d #距離
        self.w = w #道幅
        
#ノードとリンクをインスタンス化してdictに格納
dict_link = {} #ノードのdict
dict_node = {} #リンクのdict
for link in df_link.values: #全リンクのインスタンス化
    dict_link[int(link[0])] = Link(int(link[0]), int(link[1]), int(link[2]), link[3], link[4]) #インスタンス化
for node in df_node.values: #全ノードのインスタンス化
    link_ids = [int(node[i + 3]) for i in range(5) if pd.isnull(node[i + 3]) is False] #隣接するリンクのリスト
    dict_node[int(node[0])] = Node(int(node[0]), node[1], node[2], link_ids) #インスタンス化


In [5]:
###########################
#ダイクストラアルゴリズム(+A*アルゴリズム)実行関数
###########################

def Dijkstra(start_node, goal_node):
    
    #隣接ノードの最短距離を計算して情報（最短距離, 経由リンク）を更新する関数--◆--◆--◆--◆--◆--◆--◆ #・・・①
    def UpdateNodeProp(scan_node):
        for lid in scan_node.link_ids: #ノードの各隣接リンクIDについて
            link = dict_link[lid] #隣接リンク
            near_node = dict_node[link.nid2] if link.nid1== scan_node.id else dict_node[link.nid1] #隣接ノード
            
            #隣接ノードまでの合計距離が元の最短距離より短い場合は最短距離、経由リンクIDのリストを更新
            if scan_node.total_d + link.d  < near_node.total_d:
                near_node.total_d = scan_node.total_d + link.d 
                if app.bln.get(): #A*の時は最短距離+ゴールまでの直線距離を計算してtotal_d2を更新
                    l = math.sqrt((goal_node.x - near_node.x)**2 + (goal_node.y - near_node.y)**2)
                    near_node.total_d2 = near_node.total_d + l
                    heapq.heappush(list_node_temp,(near_node.total_d2,near_node.id)) #仮確定ノードのリストに追加
                else: heapq.heappush(list_node_temp,(near_node.total_d,near_node.id)) #仮確定ノードのリストに追加
                near_node.passed_links = (copy.copy(scan_node.passed_links))
                near_node.passed_links.append(link.id) #経由リンクのリストを更新
                
            #探索済みリンクを描画する場合は今回のリンクを探索済みリンクのリストに追加
            global dict_link_comp
            if app.bln2.get(): dict_link_comp.append(lid)
        
    #仮確定ノードの中で最短経路をもつノードを確定させて次のscan_nodeを返す関数--◆--◆--◆--◆--◆ #・・・②
    def FindMinNode():
        distance, nid = heapq.heappop(list_node_temp)
        next_scan_node = dict_node[nid]
        return next_scan_node
    
    #実行部分--◆--◆--◆--◆--◆-◆--◆--◆--◆--◆-◆--◆--◆--◆--◆-◆--◆--◆--◆--◆-◆--◆--◆
    
    global count, dict_link_comp
    count = 1 #確定ノード数
    dict_link_comp = [] #探索済みリンクのリスト
    list_node_temp = [] #仮確定ノードのリスト
    
    #ノードの状態の初期化（全ノードで距離=無限大,経由リンクは空にする）
    for node in dict_node.values():
        node.total_d, node.total_d2 = float('inf'), float('inf')
        node.passed_links = []
        
    #始点ノードを距離=0として,最初のscan_nodeに設定
    start_node.total_d = 0
    scan_node = start_node
    
    #goal_nodeが確定するまで探索を実行
    while goal_node != scan_node:
        UpdateNodeProp(scan_node) #・・・①
        scan_node = FindMinNode() #・・・②
        count += 1

In [6]:
###########################
#巡回地点間の距離行列と経由リンクlistの行列を算出する関数
###########################

def CalculatDistanceMatrix():
    
    #ダイクストラアルゴリズムで距離行列と経由リンクlistの行列を一列分算出する関数
    def ApplyToDijkstra(i): #・・・●
        d_mat_i, l_mat_i = [], []
        for j, key in enumerate(origin_nodes_2):
            if j < i: 
                Dijkstra(dict_node[origin_nodes_2[i]], dict_node[key]) #ダイクストラアルゴリズムの実行
                d_mat_i.append(dict_node[key].total_d) #計算した距離をリストに追加
                l_mat_i.append(dict_node[key].passed_links) #経由したリンクのリストををリストに追加
            else: d_mat_i.append(0), l_mat_i.append(0)
        return d_mat_i, l_mat_i
    
    distance_matrix = np.zeros((n_num+2, n_num+2), dtype=np.float32) #空の距離行列
    passed_links_matrix = np.empty((n_num+2, n_num+2), dtype=np.object) #空の経由リンクlistの行列
    for i in tqdm(range(n_num+2)): #まずは下三角行列を算出
        app.var_start(n_num+2) #プログレスバー
        d_mat_i, l_mat_i  = ApplyToDijkstra(i) #・・・●
        distance_matrix[i,], passed_links_matrix[i,] = np.array(d_mat_i), np.array(l_mat_i)
    i_toper = np.triu_indices(n_num+2, 1) #(n_num+2)×(n_num+2)の上三角行列のインデックス
    distance_matrix[i_toper] = distance_matrix.T[i_toper] #下三角行列を上三角行列にコピー
    passed_links_matrix[i_toper] = passed_links_matrix.T[i_toper] #下三角行列を上三角行列にコピー
    
    return distance_matrix, passed_links_matrix

In [7]:
###########################
#遺伝的アルゴリズム実行関数
###########################

def GeneticAlgorithm():
    #ノード数×個体数のランダムな行列=第一世代を生成--◆--◆--◆--◆--◆--◆--◆ #・・・①
    def generate_init_genes():
        genes = np.zeros((i_num, n_num), dtype=np.int64) #ノード数×個体数の空行列
        for i in range(i_num): genes[i,] = random.sample(range(n_num), k=n_num) #ランダムな個体を個体数生成
        return genes #地点の数×個体数の行列を出力


    #計算済みの距離行列を参照して、各個体の距離を算出する関数--◆--◆--◆--◆--◆--◆--◆ #・・・②'
    def sum_path(gene):
        d = 0
        for j in range(n_num-1): d += distance_matrix[int(gene[j])][int(gene[j + 1])]
        d += distance_matrix[-2][int(gene[0])] #スタートから最初のノードまでの距離
        if app.bln3.get():  d += distance_matrix[int(gene[-1])][-2] #最後のノードからスタートまでの距離
        else: d += distance_matrix[int(gene[-1])][-1] #最後のノードからゴールまでの距離
        return d

    #全個体の距離をリストで返す関数--◆--◆--◆--◆--◆--◆--◆ #・・・②
    def genes_path(genes):
        path_length = np.zeros(len(genes), dtype=np.float64)  #各個体の経路長を格納するリスト(空)
        for i in range(len(genes)): path_length[i] = sum_path(genes[i]) #・・・②'
        return path_length

    #確率のテーブルを元に交叉を行う個体を2つルーレット選択する関数--◆--◆--◆--◆--◆--◆--◆ #・・・③
    def roulette_choice(fitness):
        total = np.sum(fitness) #全個体の距離の合計
        roulette = np.zeros(len(fitness)) #個体ごとの確率の空テーブル
        for i in range(len(fitness)): roulette[i] = fitness[i]/total  #各個体の確率をテーブルに追加
        choiced = np.random.choice(len(roulette), 2, replace=False, p=roulette)
        #(個体数, 選ぶ数字の数, replace=重複を許すか否か, p=各々が選ばれる確率)
        return choiced

    #部分交叉--◆--◆--◆--◆--◆--◆--◆ #・・・④
    def partial_crossover(parent1, parent2):
        cross_point = random.randrange(1, n_num-1) #ランダムでtargetの初期位置を決める
        child1, child2 = parent1, parent2
        for i in range(n_num - cross_point): #初期位置から右端まで繰り返し
            target_idx = cross_point + i #１つ右にずらす
            #target_idxの値と同じ値の対配列のidxを取得
            target_1, target_2 = parent1[target_idx], parent2[target_idx]
            change_idx1,change_idx2 = np.where(parent1 == target_2), np.where(parent2 == target_1)
            #対応する値を入れ替える
            child1[target_idx], child2[target_idx] = target_2, target_1
            child1[change_idx1], child2[change_idx2] = target_1, target_2
        return child1, child2

    #突然変異（転座…ランダムで選んだ2つの要素を交換)--◆--◆--◆--◆--◆--◆--◆ #・・・⑤
    def translocation_mutation(genes, m_num, p_value):
        m_genes = genes
        for i in range(m_num):
            if np.random.choice(2, 1, p = [1-p_value, p_value]) == 1:  #一定確率で発動
                m_value = np.random.choice(genes[i], 2, replace = False) #要素をランダムで2つ選択
                m_idx1, m_idx2 = np.where(genes[i] == m_value[0]), np.where(genes[i] == m_value[1]) #選択要素のidxを取得
                m_genes[i][m_idx1], m_genes[i][m_idx2] = m_value[1], m_value[0] #２つの要素を入れ替え
        return m_genes

    #実行部分--◆--◆--◆--◆--◆--◆--◆
    
    t1 = time.time()
    genes = generate_init_genes() #ノード数×個体数のランダムな行列=第0世代を生成 #・・・①
    global min_path_length
    min_path_length = float('inf') #最短の経路長を初期化
    text1 = ""
    
    #設定した世代数までループを実行
    for i in tqdm(range(generation)):
        if i%100 == 0: app.var_start(generation/100) #プログレスバーの表示用
            
        #次世代の空行列の作成
        child = np.zeros(np.shape(genes),  dtype=np.int8)
        
        #個体ごとの適合度(距離の逆数)を算出
        path_length = genes_path(genes) #各個体の経路長のリスト　#・・・②
        fitness = np.reciprocal(path_length) #各個体の適合度のリスト
        
        #まずはエリート選択(適応度の高いものから順番に選択)
        for j in range(i_num-elite, i_num): child[j] = genes[np.argsort(fitness)[j]]
            
        #残りはルーレット選択→交叉
        for j in range(int((i_num-elite)/2)):
            parents = roulette_choice(fitness) #ルーレット選択を実行して2個体を選択　#・・・③
            child[2*j], child[2*j+1] = partial_crossover(genes[parents[0]], genes[parents[1]]) #部分交叉 #・・・④
            
        #一定確率で突然変異(エリート選択した個体は突然変異の対象外) #・・・⑤
        child = translocation_mutation(child, i_num-elite, p_mutation)
        
        #世代交代
        genes = child

        #世代のトップ個体がハイスコアを出した場合は最短経路長を更新
        if min(path_length) < min_path_length:
            min_path_length = min(path_length)
            if renzoku == False: #10連続実行のときは実行しない
                text1 = text1 + '<第' + str(i+1) + '世代> :' + str(int(min(path_length))) + '[m]\n'
                app.text.set(text1)
                print('<第' + str(i+1) + '世代> :' + str(min(path_length)) + '[m]')
        
    t2 = time.time()
    if renzoku == False: #10連続実行のときは実行しない
        text1 = text1 +'\n最短距離：' + str(int(min(path_length))) + '[m]\n' + '（実行時間：' + str(round(t2 - t1, 3)) + '[s]）'
        app.text.set(text1)
        print('<最短距離> :' + str(min(path_length)) + '[m]')
    
    return genes[np.argmin(path_length)]


In [8]:
###########################
#GUI用作成用のクラスとGUI操作に必要な関数群
###########################

import tkinter as tk
import tkinter.ttk as ttk
from tkinter import messagebox

class MyApp1(tk.Frame):
    
    def __init__(self, master=None):
        super().__init__(master)
        self.pack()
        
        #キャンバスを入れるフレームを作成
        self.frame = tk.Frame(root,width=canvas_size[0],height=canvas_size[1])
        self.frame.pack(side=tk.LEFT, expand=True, fill=tk.BOTH)
        #キャンバスを作成
        self.canvas = tk.Canvas(self.frame, width=canvas_size[0], height=canvas_size[1],
                                scrollregion=(-canvas_size[0]/4,-canvas_size[1]/4,canvas_size[0]*5/4,canvas_size[1]*5/4))
        self.Draw_map() #キャンバスに図面を描画
        #スクロールバーを作成
        self.hbar=ttk.Scrollbar(self.frame,orient=tk.HORIZONTAL)
        self.hbar.pack(side=tk.BOTTOM,fill=tk.X)
        self.hbar.config(command=self.canvas.xview)
        self.vbar=ttk.Scrollbar(self.frame,orient=tk.VERTICAL)
        self.vbar.pack(side=tk.RIGHT,fill=tk.Y)
        self.vbar.config(command=self.canvas.yview)
        self.canvas.config(xscrollcommand=self.hbar.set, yscrollcommand=self.vbar.set)
        #プログレスバーの作成
        self.progressbar = ttk.Progressbar(root, orient=tk.HORIZONTAL, length=200, mode='determinate')
        self.progressbar.pack(side="bottom")
        #クリックイベントの作成
        self.canvas.bind('<Double-Button-1>', self.mouse_canvas) #キャンバス上でダブルクリックされたときのイベントを設定
        #self.canvas.bind('<MouseWheel>', self.Zoom_Map) #キャンバス上で左クリックされたときのイベントを設定
        self.canvas.bind("<ButtonPress-1>", self.Start_Scroll) #ドラッグでスクロール
        self.canvas.bind("<B1-Motion>", self.Scroll) #ドラッグでスクロール
        self.canvas.bind("<ButtonRelease-1>", self.End_Scroll) #ドラッグでスクロール
        self.canvas.pack(expand=True, fill=tk.BOTH)
        # チェックボタン作成
        self.bln = tk.BooleanVar()
        self.bln2 = tk.BooleanVar()
        self.bln3 = tk.BooleanVar()
        self.bln.set(True)
        self.bln2.set(False)
        self.bln3.set(False)
        #ラベルの作成
        self.text1 = tk.StringVar()
        self.text = tk.StringVar()
        self.text1.set("")
        self.text.set("　　　　　　　　　　")
        # ボタンとかの配置
        self.chk = tk.Checkbutton(root, bg = 'lemon chiffon', variable=self.bln, text='A*アルゴリズムを使用').pack()
        self.words = tk.Label(root, bg = 'lemon chiffon', textvariable=self.text1, font=("", fsize)).pack() #空行を挿入
        self.button1 = tk.Button(root, width=bwidth, text = 'スタート地点を設定', fg = "Lightblue3", command = self.button1_click).pack()
        self.button2 = tk.Button(root, width=bwidth, text = 'ゴール地点を設定', fg = "Lightpink3", command = self.button2_click).pack()
        self.button3 = tk.Button(root, width=bwidth, text = 'ランダムで２地点を設定', command = self.button3_click).pack()
        self.button4 = tk.Button(root, width=bwidth, text='２地点の最短経路を算出', fg = "red", command=self.button4_click).pack()
        self.chk2 = tk.Checkbutton(root, bg = 'lemon chiffon', variable=self.bln2, text='探索済みリンクを描画').pack() # チェックボタン作成
        self.words = tk.Label(root, bg = 'lemon chiffon', textvariable=self.text1, font=("", fsize)).pack() #空行を挿入
        self.button5 = tk.Button(root, width=bwidth, text = 'ランダムで巡回地点を設定', command = self.button5_click).pack()
        self.button6 = tk.Button(root, width=bwidth, text = '距離行列を算出', command = self.button6_click).pack()
        self.button7 = tk.Button(root, width=bwidth, text='遺伝的アルゴリズム - 実行', fg = "orange", command=self.button7_click).pack()
        self.button8 = tk.Button(root, width=bwidth, text='10連続で実行', fg = "orange", command=self.button8_click).pack()
        self.chk3 = tk.Checkbutton(root, bg = 'lemon chiffon', variable=self.bln3, text='循環ルートを検索').pack() # チェックボタン作成
        self.words = tk.Label(root, bg = 'lemon chiffon', textvariable=self.text1, font=("", fsize)).pack() #空行を挿入
        self.button_zoom_in = tk.Button(root, width=bwidth, text = '拡大', command = self.Zoom_In).pack()
        self.button_zoom_out = tk.Button(root, width=bwidth, text = '縮小', command = self.Zoom_Out).pack()
        self.words = tk.Label(root, bg = 'lemon chiffon', textvariable=self.text1, font=("", fsize)).pack() #空行を挿入
        self.words = tk.Label(root, textvariable=self.text, font=("", fsize)).pack()
        
    #プログレスバーの更新
    def var_start(self, maximum_bar):
        global value_bar
        value_bar += 1
        self.progressbar.configure(maximum=maximum_bar, value=value_bar)
        self.progressbar.update()
        if value_bar==maximum_bar:
            self.progressbar.stop()
            value_bar = 0
    
    #ドラッグで画面スクロール    
    def Start_Scroll(self, event):
        global is_clicking, before_x, before_y
        is_clicking = True # クリック中フラグを立てる
        before_x, before_y = event.x, event.y # 現在のマウスの位置を記憶
    def Scroll(self, event):
        global before_x, before_y
        if is_clicking:
            dx, dy = event.x - before_x, event.y - before_y # 前回からのマウスのピクセル数を取得
            h, v = self.hbar.delta(dx, dy), self.vbar.delta(dx, dy) # 「マウスの移動量」を取得
            start_x, end_x = self.hbar.get() # 水平方向の「現在のキャンバスのスクロール位置」を取得
            start_y, end_y = self.vbar.get() # 垂直方向の「現在のキャンバスのスクロール位置」を取得
            after_x = max(min(start_x - h, 1.0), 0.0)  # 水平方向の「移動後のキャンバスのスクロール位置」を取得# 値は0〜1に丸める
            after_y = max(min(start_y - v, 1.0), 0.0) # 垂直方向の「移動後のキャンバスのスクロール位置」を取得 # 値は0〜1に丸める
            self.canvas.xview_moveto(after_x) # 水平方向に対して「移動後のキャンバスのスクロール位置」にスクロール
            self.canvas.yview_moveto(after_y) # 垂直方向に対して「移動後のキャンバスのスクロール位置」にスクロール
            before_x, before_y = event.x, event.y # 現在のマウスの位置を記憶
    def End_Scroll(self, event):
        global is_clicking
        is_clicking = False # クリック中フラグを下ろす
    
    #図面の拡大縮小
    def Zoom_In(self): #拡大ボタン
        global zoom
        zoom *= 1.5
        self.canvas.delete('select', 'start', 'goal', 'result', 'select_GA', 'result_GA')
        self.canvas.config(scrollregion=(-canvas_size[0]*zoom/4,-canvas_size[1]*zoom/4,canvas_size[0]*zoom*5/4,canvas_size[1]*zoom*5/4))
        self.Draw_map()
    def Zoom_Out(self): #縮小ボタン
        global zoom
        zoom /= 1.5
        self.canvas.delete('select', 'start', 'goal', 'result', 'select_GA', 'result_GA')
        self.canvas.config(scrollregion=(-canvas_size[0]*zoom/4,-canvas_size[1]*zoom/4,canvas_size[0]*zoom*5/4,canvas_size[1]*zoom*5/4))
        self.Draw_map()
    def Zoom_Map(self, event): #ホイールで拡大縮小
        global zoom, keep_delta
        keep_delta += event.delta
        if keep_delta > -1200:
            zoom *= (100+keep_delta/12) / 100
            self.canvas.delete('select', 'start', 'goal', 'result', 'select_GA', 'result_GA')
            self.Create_Canvas()
            self.Draw_map()
    
    #図面の描画
    def Draw_map(self):
        self.canvas.delete('map')
        for link in dict_link.values(): self.Draw_link(link, 'gray50', link.w*ratio, 'map') #全リンクの描画
    #リンクの描画
    def Draw_link(self, link, color, w, tag):
        x1, y1 = (dict_node[link.nid1].x - vertex[0]) * ratio * zoom, (dict_node[link.nid1].y - vertex[1]) * ratio * zoom
        x2, y2 = (dict_node[link.nid2].x - vertex[0]) * ratio * zoom, (dict_node[link.nid2].y - vertex[1]) * ratio * zoom
        self.canvas.create_line(x1, y1, x2, y2, fill = color, width = w * zoom, tag = tag) 
    #ノードの描画
    def Draw_node(self, node, color, size, tag):
        x1, y1 = ((node.x - size) - vertex[0]) * ratio * zoom, ((node.y - size) - vertex[1]) * ratio * zoom
        x2, y2 = ((node.x + size) - vertex[0]) * ratio * zoom, ((node.y + size) - vertex[1]) * ratio * zoom
        self.canvas.create_oval(x1, y1, x2, y2, fill = color, tag = tag)
       
    #左クリックした点から最も近いノードを選択
    def mouse_canvas(self, event):
        global select_nid
        click_x, click_y = (self.canvas.canvasx(event.x) / ratio) / zoom + vertex[0], (self.canvas.canvasy(event.y) / ratio) / zoom + vertex[1]
        l, nid =  float('inf'), None
        for node in dict_node.values(): #各ノードまでの距離を比較して最も近いノードを選択
            l_ = ((node.x - click_x)**2) + ((node.y - click_y)**2)
            if l_ < l: l, nid = l_, node.id
        select_nid = nid
        self.canvas.delete('select', 'result', 'select_GA', 'result_GA') #古いselectノード等を削除
        self.Draw_node(dict_node[select_nid], 'blue', psize, 'select') #新しく選択したノードを描画
        
    #「スタート地点を設定」ボタン
    def button1_click(self):
        if select_nid == None: messagebox.showerror("エラー", 'ノードを選択してください！')
        else:
            global start_nid
            start_nid = select_nid
            self.canvas.delete('select', 'start', 'result', 'select_GA', 'result_GA')
            self.Draw_node(dict_node[start_nid], 'Lightblue1', psize, 'start')
    #「ゴール地点を設定」ボタン
    def button2_click(self):
        if select_nid == None: messagebox.showerror("エラー", 'ノードを選択してください！')
        else:
            global goal_nid
            goal_nid = select_nid
            self.canvas.delete('select', 'goal', 'result', 'select_GA', 'result_GA')
            self.Draw_node(dict_node[goal_nid], 'Lightpink1', psize, 'goal')  
    #「ランダムで２地点を設定」ボタン
    def button3_click(self):
        global start_nid, goal_nid
        start_goal = random.sample(list(dict_node.keys()), 2) #2地点のノードIDを格納したリスト
        start_nid, goal_nid = start_goal[0], start_goal[1]
        self.canvas.delete('select', 'start', 'goal', 'result', 'select_GA', 'result_GA')
        self.Draw_node(dict_node[start_nid], 'Lightblue1', psize, 'start')
        self.Draw_node(dict_node[goal_nid], 'Lightpink1', psize, 'goal')
        
    #「２地点の最短経路を算出」ボタン
    def button4_click(self):
        if start_nid == None or goal_nid == None: messagebox.showerror("エラー", 'StartとGoalを設定してください！')
        else:
            t1 = time.time()
            Dijkstra(dict_node[start_nid], dict_node[goal_nid]) #実行
            self.canvas.delete('select', 'start', 'goal', 'result', 'select_GA', 'result_GA')
            if self.bln2.get(): #探索済みリンクの描画
                for lid in dict_link_comp: self.Draw_link(dict_link[lid], 'tan1', rwidth-3, 'result')
            for lid in dict_node[goal_nid].passed_links: #結果の描画
                self.Draw_link(dict_link[lid], 'coral1', rwidth, 'result')
                self.Draw_node(dict_node[start_nid], 'Lightblue1', psize, 'start')
                self.Draw_node(dict_node[goal_nid], 'Lightpink1', psize, 'goal')
            t2 = time.time()
            print('<経路探索finish> 距離：' + str(round(dict_node[goal_nid].total_d, 2)) + '[m]\n' + 
                  '（実行時間：' + str(round(t2 - t1, 3)) + '[s]）')
            self.text.set('最短距離：' + str(round(dict_node[goal_nid].total_d, 2)) + '[m]\n' 
                          + '探索ノード数：' + str(count) + '\n'  + '実行時間：' + str(round(t2 - t1, 3)) + '[s]\n')  
        
    #「ランダムで巡回地点を設定」ボタン
    def button5_click(self):
        if start_nid == None or goal_nid == None: messagebox.showerror("エラー", 'StartとGoalを設定してください！')
        else:
            global origin_nodes, origin_nodes_2, distance_matrix, passed_links_matrix
            origin_nodes = random.sample(list(dict_node.keys()), n_num) #n地点のノードIDを格納したリスト
            self.canvas.delete('select', 'start', 'goal', 'result', 'select_GA', 'result_GA')
            distance_matrix, passed_links_matrix = None, None #距離行列と経由リンクlistの行列を初期化
            for nid in origin_nodes: self.Draw_node(dict_node[nid], 'gold', psize, 'select_GA')
            self.Draw_node(dict_node[start_nid], 'Lightblue1', psize, 'select_GA')
            self.Draw_node(dict_node[goal_nid], 'Lightpink1', psize, 'select_GA')
            origin_nodes_2 = copy.copy(origin_nodes) #スタートとゴールも含めた巡回地点
            origin_nodes_2.extend([start_nid, goal_nid])
            print('<選択されたノードID>\n' + str(origin_nodes) + '\n ') 
        
    #「距離行列(と経由リンクlistの行列)を算出」ボタン
    def button6_click(self):
        if origin_nodes == None: messagebox.showerror("エラー", '巡回地点を選択してください！')
        else:
            t1 = time.time()
            global distance_matrix, passed_links_matrix
            distance_matrix, passed_links_matrix = CalculatDistanceMatrix()
            self.canvas.delete('select', 'start', 'goal', 'result', 'select_GA', 'result_GA')
            for nid in origin_nodes: self.Draw_node(dict_node[nid], 'orange', psize, 'select_GA')
            self.Draw_node(dict_node[start_nid], 'Lightblue1', psize, 'select_GA')
            self.Draw_node(dict_node[goal_nid], 'Lightpink1', psize, 'select_GA')
            t2 = time.time()
            self.text.set("距離行列を算出しました。\n" + '（実行時間：' + str(round(t2 - t1, 3)) + '[s]）')
            print('<距離行列>\n' + str(distance_matrix) + '\n（実行時間：' + str(round(t2 - t1, 3)) + '[s]）\n ')
            
    def GA_Draw(self, g): #遺伝的アルゴリズムの結果描画
        self.canvas.delete('select', 'start', 'goal', 'result', 'select_GA', 'result_GA')
        for i in range(n_num-1):
            lids = passed_links_matrix[g[i]][g[i+1]]
            for lid in lids: self.Draw_link(dict_link[lid], 'coral1', rwidth, 'result_GA')
        lids = passed_links_matrix[-2][g[0]]
        for lid in lids: self.Draw_link(dict_link[lid], 'coral1', rwidth, 'result_GA')
        if self.bln3.get(): 
            lids = passed_links_matrix[g[-1]][-2]
            for lid in lids: self.Draw_link(dict_link[lid], 'coral1', rwidth, 'result_GA')
            self.Draw_node(dict_node[start_nid], 'Lightblue1', psize, 'select_GA')
        else:
            lids = passed_links_matrix[g[-1]][-1]
            for lid in lids: self.Draw_link(dict_link[lid], 'coral1', rwidth, 'result_GA')
            self.Draw_node(dict_node[start_nid], 'Lightblue1', psize, 'select_GA')
            self.Draw_node(dict_node[goal_nid], 'Lightpink1', psize, 'select_GA')
        for i, n in enumerate(g):
            node = dict_node[origin_nodes[n]]
            self.Draw_node(node, 'orange', psize, 'select_GA')
            x, y = (node.x - vertex[0]) * ratio * zoom, (node.y - vertex[1]) * ratio * zoom
            self.canvas.create_text(x, y, text = int(i + 1), font = ('Arial', tsuze), fill = 'white', tag = 'result_GA')
        
    #「遺伝的アルゴリズム実行」ボタン
    def button7_click(self):
        if origin_nodes == None: messagebox.showerror("エラー", '巡回地点を選択してください！')
        else:
            t1 = time.time()
            self.GA_Draw(GeneticAlgorithm())
            t2 = time.time()
            print('<経路探索finish>\n' + '（実行時間：' + str(round(t2 - t1, 3)) + '[s]）')
    
    #「10連続で実行」ボタン
    def button8_click(self):
        global renzoku, min_path_length
        renzoku = True
        text_ = ""
        if origin_nodes == None: messagebox.showerror("エラー", '巡回地点を選択してください！')
        else:
            t1 = time.time()
            best_g, best_path_length = None, float('inf')
            for i in range(10):
                g = GeneticAlgorithm()
                if min_path_length <= best_path_length: best_path_length, best_g = min_path_length, g
                text_ = text_ + '<' + str(i+1) + '回目> :' + str(int(min_path_length)) + '[m]\n'
                self.text.set(text_)
            self.GA_Draw(best_g)
            renzoku = False
            t2 = time.time()
            text_ = text_ +'\n最短距離：' + str(int(best_path_length)) + '[m]\n' + '（実行時間：' + str(round(t2 - t1, 3)) + '[s]）'
            self.text.set(text_)

In [9]:
###########################
#グローバル変数＋各種パラメータ
###########################

#ダイクストラアルゴリズム用
#グローバル変数
select_nid = None #選択(クリック中)ノードのID
start_nid = None #始点ノードのID
goal_nid = None #終点ノードのID
count = 1 #確定ノード数(始点は最初から確定なのでcount=1)
dict_link_comp = []#探索済みリンクidのリスト

#遺伝的アルゴリズム用
#グローバル変数
origin_nodes = None #ランダムで選択したノードIDを格納
origin_nodes_2 = None #スタートとゴールも含めた巡回地点
distance_matrix = None #距離行列
passed_links_matrix = None #経由リンクlistの行列
renzoku = False #10連続実行=True
min_path_length = float('inf') #最短経路長
#パラメータ
n_num = 10 #ノード数
i_num = 30 #一世代の個体数
generation = 100 #世代数
elite = 4 #エリート選択数(個体数-エリート選択数は偶数)
p_mutation = 0.01 #突然変異率

#描画用
#グローバル変数
is_clicking = False
zoom = 1 #canvasの倍率
before_x = 0
before_y = 0
value_bar = 0 #プログレスバー
keep_delta = 0
#パラメータ
psize = 20 #描画する選択ノードの大きさ
rwidth = 5 #結果を描画するときのリンクの太さ
tsuze = 10 #描画するテキストの大きさ
bwidth = 20 #ボタンの横幅
fsize = 12 #実行結果のフォントサイズ
#メモ：描画のtag…[map, select, start, goal, result, select_GA, result_GA]

In [10]:
###########################
#アプリの実行
###########################

root = tk.Tk()
root.geometry("{}x{}+0+0".format(int(canvas_size[0]+200), int(canvas_size[1]+300))) #(0,0)はwindowのスクリーン上の配置位置
root.title('経路探索') #タイトル作成
root.configure(background = 'lemon chiffon')
app = MyApp1(master=root)
app.mainloop()